## This notebook

This notebook is based on [NFL Baseline - Simple Helmet Mapping](https://www.kaggle.com/its7171/nfl-baseline-simple-helmet-mapping) by [tito](https://www.kaggle.com/its7171).  

The major change is to select trackings according to the number of helmets.  
In tito's notebook, itertools.combinations was used to randomly select what not to use. There are some combinations that cannot be verified with this method. Another problem is that there is a trade-off between execution speed and accuracy.

In this implementation, the distance is measured between each player and the player with the close distance is selected. This probably improved the execution speed (The speed is almost 28 times faster!).

The reason why I decided on this idea is because I thought that what would fit in the picture would be a close distance between the players, and players who are far apart would not be captured in the first place. The implementation itself uses sklearn.neighbors.KDTree, but since this is the first time I have used it, there may be other better ways to handle it.

There are also some minor changes, such as using scipy.optimize.minimize_scalar to search for DIG, and the normalize_arr function is different.  

But I haven't tested which process is better, so if anyone has, I'd appreciate it if you'd share!
If you have any questions or suggestions for improvement, please comment.

## Reference

* [NFL Baseline - Simple Helmet Mapping](https://www.kaggle.com/its7171/nfl-baseline-simple-helmet-mapping) by [tito](https://www.kaggle.com/its7171).  
* [Helper Code + Helmet Mapping + Deepsort](https://www.kaggle.com/robikscube/helper-code-helmet-mapping-deepsort) by [
Rob Mulla](https://www.kaggle.com/robikscube)

## IMPORT MODULE & CONFIG

In [1]:
import os
import sys
import random
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import cv2
import itertools
from pathlib import Path
from glob import glob
from tqdm.notebook import tqdm
from multiprocessing import Pool

from scipy.optimize import minimize, minimize_scalar
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KDTree

In [3]:
from external_lib.NFLlib.score import NFLAssignmentScorer, check_submission
from external_lib.NFLlib.features import add_track_features


In [4]:
# Config
SEED = 42
CONF_THRE = 0.30
DIG_MAX = 30

## MAIN

In [11]:
def add_cols(df):
    df['game_play'] = df['video_frame'].str.split('_').str[:2].str.join('_')
    if not 'video' in df.columns:
        df['video'] = df['video_frame'].str.split('_').str[:3].str.join('_') + '.mp4'
    return df


# path 
BASE_DIR = Path('/work/data/input/nfl-health-and-safety-helmet-assignment')
TRAIN_DIR = BASE_DIR / 'train'
TEST_DIR = BASE_DIR / 'test'
IMAGES_DIR = BASE_DIR / 'images'
    
# Reading data
DEBUG = (len(glob(str(TEST_DIR / '*'))) == 6)

if DEBUG:
    tracking = pd.read_csv(BASE_DIR / 'train_player_tracking.csv')
    helmets = pd.read_csv(BASE_DIR / 'train_baseline_helmets.csv')
else:
    tracking = pd.read_csv(BASE_DIR / 'test_player_tracking.csv') 
    helmets = pd.read_csv(BASE_DIR / 'test_baseline_helmets.csv')
    
labels = pd.read_csv(BASE_DIR / 'train_labels.csv')
sub = pd.read_csv(BASE_DIR / 'sample_submission.csv')

# processing data
tracking = add_track_features(tracking)
helmets = add_cols(helmets)
labels = add_cols(labels)

# sampling Data
if False:
    sample_videos = labels['video'].drop_duplicates().sample(1, random_state=42).tolist()
    sample_gameplays = ['_'.join(x.split('_')[:2]) for x in sample_videos]
    tracking = tracking[tracking['game_play'].isin(sample_gameplays)]
    helmets = helmets[helmets['video'].isin(sample_videos)]
    labels = labels[labels['video'].isin(sample_videos)]

print(tracking.shape, helmets.shape, labels.shape)

(333811, 18) (1225172, 8) (952087, 15)


In [27]:
def find_nearest(arr: np.array, value: int):
    arr, value = arr.astype(int), int(value)
    idx = np.abs(arr - value).argmin()
    return arr[idx]

def normalize_arr(arr: np.array):
    _mean = np.mean(arr, axis=0)
    out_arr = arr - _mean
    _norm = np.linalg.norm(out_arr)
    out_arr = out_arr / _norm
    return out_arr

def rotate_arr(u, t):
    t = np.deg2rad(t)
    R = np.array([
        [np.cos(t), -np.sin(t)],
        [np.sin(t),  np.cos(t)]
    ])
    return  np.dot(R, u.T).T

def mapping(h_arr: np.array, t_arr: np.array):
    out_norm = float('INF')
    out_idx = None
    out_x = None
    out_dig = None
    
    tree = KDTree(t_arr)
    for i in range(len(t_arr)):
        dist, idx = tree.query([t_arr[i]], k=len(h_arr))
        idx = np.sort(idx[0])
        norm_t_arr = normalize_arr(t_arr[idx])
                
        # minimization norm
        def opt_rot(dig):
            rot_t_arr = rotate_arr(norm_t_arr, dig)
            print("----------------------")
            display(rot_t_arr)
            display(h_arr)
            return np.linalg.norm(np.sort(rot_t_arr[:, 0])-h_arr[:, 0])
                
        for bounds in [(-DIG_MAX , DIG_MAX), (180-DIG_MAX, 180+DIG_MAX)]:
            result = minimize_scalar(opt_rot, bounds=bounds, method='bounded') 
            if out_norm > result.fun:
                out_norm = result.fun
                out_idx = idx
                out_x = rotate_arr(norm_t_arr, result.x)[:, 0]
                out_dig = result.x
                
    return out_idx, out_x

def main(args: pd.DataFrame):
    video_frame, subhelmets = args
    gameKey, playID, view, frame = video_frame.split('_')
    gameKey, playID, frame = int(gameKey), int(playID), int(frame)

    # get nearest-frame
    _index = (tracking['gameKey']==gameKey) & (tracking['playID']==playID)
    subtracking = tracking[_index].copy()
    est_frame = find_nearest(subtracking["est_frame"].values, frame)
    subtracking = subtracking[subtracking['est_frame']==est_frame]
    subtracking = subtracking.reset_index(drop=True)

    if view == 'Endzone':
        subtracking[['x', 'y']] = subtracking[['y', 'x']].values

    # normalizing 
    subhelmets = subhelmets[subhelmets['conf']>CONF_THRE].copy()

    if len(subhelmets) > len(subtracking):
        subhelmets = subhelmets.tail(len(subtracking))

    subhelmets['x'] = subhelmets['left'] + subhelmets['width'] // 2
    subhelmets['y'] = subhelmets['top'] + subhelmets['height'] // 2
    subhelmets[['norm_x', 'norm_y']] = normalize_arr(subhelmets[['x', 'y']].values)
    subhelmets = subhelmets.sort_values('norm_x').reset_index(drop=True)

    # mapping tracking2helmets
    h_arr = subhelmets[['norm_x', 'norm_y']].values
    t_arr = subtracking[['x', 'y']].values
    out_idx, out_x = mapping(h_arr, t_arr)

    # helmets labeling 
    players = subtracking['player'].tolist()
    players = [p for i, p in enumerate(players) if i in out_idx]
    _pred = pd.DataFrame({'label': players, 'x': out_x}).sort_values('x')['label']
    subhelmets['label'] = _pred.values
    
    return subhelmets[['video_frame', 'left', 'width', 'top', 'height', 'label']]

----------------------


array([[ 1.94069939e-01, -4.32170917e-03],
       [-1.07907362e-01,  2.92323704e-02],
       [ 3.09757374e-01, -2.61252431e-02],
       [-3.36737290e-01,  2.56805318e-01],
       [-7.30329086e-02,  2.78039769e-01],
       [ 6.38224227e-02, -2.91494492e-05],
       [-1.57211946e-01,  1.58707974e-01],
       [ 1.42575632e-01,  1.99237842e-01],
       [ 5.86184975e-03,  2.07306050e-01],
       [ 7.28599081e-02, -9.87421810e-02],
       [-2.59691038e-01, -3.21621542e-02],
       [ 1.08272531e-02, -7.17154586e-02],
       [-6.07597981e-02, -6.97569542e-02],
       [-1.27138526e-01, -1.01636109e-01],
       [-1.89407674e-01, -8.44876423e-02],
       [ 2.40299436e-01, -9.87385378e-02],
       [ 1.50719504e-01, -9.00861923e-02],
       [ 3.58272831e-02, -3.96819640e-01],
       [ 8.52659408e-02, -5.47083521e-02]])

array([[-0.38036206,  0.03186022],
       [-0.32787067,  0.04032658],
       [-0.2423605 ,  0.02932032],
       [-0.21442153,  0.04201985],
       [-0.15092389, -0.06634947],
       [-0.12298492,  0.14107618],
       [-0.11875174,  0.02762705],
       [-0.09335268,  0.03186022],
       [-0.057794  , -0.21281738],
       [-0.03154831,  0.04371312],
       [ 0.02348299, -0.05280331],
       [ 0.06073494,  0.04201985],
       [ 0.14539848,  0.02678041],
       [ 0.14793838,  0.05387274],
       [ 0.15725137, -0.0934418 ],
       [ 0.22074902,  0.05387274],
       [ 0.22328892, -0.0477235 ],
       [ 0.31895872,  0.00476789],
       [ 0.44256747, -0.09598171]])

----------------------


array([[ 0.18922753,  0.0432985 ],
       [-0.11178001,  0.00193879],
       [ 0.30673326,  0.05046648],
       [-0.38934027,  0.1665987 ],
       [-0.13884884,  0.25171595],
       [ 0.06188929,  0.01558905],
       [-0.19126831,  0.11541339],
       [ 0.08948777,  0.228069  ],
       [-0.04504405,  0.20243812],
       [ 0.09480706, -0.07791153],
       [-0.24392609, -0.09473067],
       [ 0.02804683, -0.06688581],
       [-0.04184314, -0.08250416],
       [-0.09840307, -0.129657  ],
       [-0.16297539, -0.1282671 ],
       [ 0.25715536, -0.03693562],
       [ 0.16818152, -0.05046648],
       [ 0.13183968, -0.37598899],
       [ 0.09606088, -0.03218062]])

...

array([[-1.88423660e-01, -4.66727185e-02],
       [ 1.11796810e-01,  5.88088239e-05],
       [-3.05782554e-01, -5.59391534e-02],
       [ 3.92254909e-01, -1.59615352e-01],
       [ 1.43324347e-01, -2.49194804e-01],
       [-6.16008605e-02, -1.66924076e-02],
       [ 1.93299987e-01, -1.11977374e-01],
       [-8.53983361e-02, -2.29631558e-01],
       [ 4.86540394e-02, -2.01600953e-01],
       [-9.61840502e-02,  7.62050721e-02],
       [ 2.42194498e-01,  9.90740339e-02],
       [-2.92374759e-02,  6.63739890e-02],
       [ 4.03622718e-02,  8.32386493e-02],
       [ 9.60706423e-02,  1.31394577e-01],
       [ 1.60657492e-01,  1.31158679e-01],
       [-2.57774271e-01,  3.23348549e-02],
       [-1.69056413e-01,  4.74533484e-02],
       [-1.38536830e-01,  3.73573243e-01],
       [-9.66205456e-02,  3.04590645e-02]])

array([[-0.38036206,  0.03186022],
       [-0.32787067,  0.04032658],
       [-0.2423605 ,  0.02932032],
       [-0.21442153,  0.04201985],
       [-0.15092389, -0.06634947],
       [-0.12298492,  0.14107618],
       [-0.11875174,  0.02762705],
       [-0.09335268,  0.03186022],
       [-0.057794  , -0.21281738],
       [-0.03154831,  0.04371312],
       [ 0.02348299, -0.05280331],
       [ 0.06073494,  0.04201985],
       [ 0.14539848,  0.02678041],
       [ 0.14793838,  0.05387274],
       [ 0.15725137, -0.0934418 ],
       [ 0.22074902,  0.05387274],
       [ 0.22328892, -0.0477235 ],
       [ 0.31895872,  0.00476789],
       [ 0.44256747, -0.09598171]])

----------------------


array([[-1.88423650e-01, -4.66727587e-02],
       [ 1.11796810e-01,  5.88327201e-05],
       [-3.05782542e-01, -5.59392188e-02],
       [ 3.92254943e-01, -1.59615268e-01],
       [ 1.43324400e-01, -2.49194773e-01],
       [-6.16008569e-02, -1.66924208e-02],
       [ 1.93300010e-01, -1.11977332e-01],
       [-8.53982870e-02, -2.29631576e-01],
       [ 4.86540824e-02, -2.01600943e-01],
       [-9.61840665e-02,  7.62050515e-02],
       [ 2.42194477e-01,  9.90740857e-02],
       [-2.92374901e-02,  6.63739828e-02],
       [ 4.03622540e-02,  8.32386579e-02],
       [ 9.60706142e-02,  1.31394598e-01],
       [ 1.60657464e-01,  1.31158713e-01],
       [-2.57774278e-01,  3.23347998e-02],
       [-1.69056423e-01,  4.74533123e-02],
       [-1.38536910e-01,  3.73573213e-01],
       [-9.66205521e-02,  3.04590439e-02]])

array([[-0.38036206,  0.03186022],
       [-0.32787067,  0.04032658],
       [-0.2423605 ,  0.02932032],
       [-0.21442153,  0.04201985],
       [-0.15092389, -0.06634947],
       [-0.12298492,  0.14107618],
       [-0.11875174,  0.02762705],
       [-0.09335268,  0.03186022],
       [-0.057794  , -0.21281738],
       [-0.03154831,  0.04371312],
       [ 0.02348299, -0.05280331],
       [ 0.06073494,  0.04201985],
       [ 0.14539848,  0.02678041],
       [ 0.14793838,  0.05387274],
       [ 0.15725137, -0.0934418 ],
       [ 0.22074902,  0.05387274],
       [ 0.22328892, -0.0477235 ],
       [ 0.31895872,  0.00476789],
       [ 0.44256747, -0.09598171]])

In [12]:
# multi processing
df_list = helmets.groupby('video_frame')
submission_df = []

submission_df = exe_multiprocess_ret_as_list(main,df_list)

# submission
submission_df = pd.concat(submission_df).reset_index(drop=True)
submission_df.to_csv('submission_original.csv', index=False)

if DEBUG:
    scorer = NFLAssignmentScorer(labels[labels['video_frame'].isin(submission_df['video_frame'].unique())])
    score = scorer.score(submission_df)
    print(f'score: {round(score, 5)}')

100%|██████████| 52141/52141 [08:49<00:00, 98.43it/s] 


score: 0.2258


In [19]:


if DEBUG:
    view_mode = "Sideline"
    #view_mode = "Endzone"
    
    scorer = NFLAssignmentScorer(labels[labels["video_frame"].str.contains(view_mode)])
    score = scorer.score(submission_df[submission_df["video_frame"].str.contains(view_mode)])
    print(f'score: {round(score, 5)}')

score: 0.17716
